<a href="https://colab.research.google.com/github/dnaae/minor-internship/blob/FISH-QUANT-pipeline/channel_splitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Imports
import os
import numpy as np
import tifffile as tiff
from google.colab import drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')
# Specify the top-level folder in your Google Drive
drive_folder = "/content/drive/MyDrive/"

In [ ]:
#Function to interactively select treatment folder
def input_folder_interactive():
  # List contents of the drive_folder
  contents = os.listdir(drive_folder)
  # Print contents and ask the user to select a folder
  print("Contents of your Google Drive:")
  for i, item in enumerate(contents):
      print(f"{i + 1}: {item}")
  try:
      choice = int(input("Enter the number of the folder you want to select: "))
      if 1 <= choice <= len(contents):
          selected_folder = os.path.join(drive_folder, contents[choice - 1])
          return selected_folder
      else:
          print("Invalid choice. Exiting.")
          return None
  except ValueError:
      print("Invalid input. Exiting.")
      return None

In [ ]:
# Function to split channels and save as separate TIFF files
def split_and_save_channels(input_folder, output_folder):
    # List all image files in the input folder
    image_files = [f for f in os.listdir(input_folder) if f.endswith(".tif")]

    for image_file in image_files:
        image_path = os.path.join(input_folder, image_file)

        # Load the image using tifffile
        image = tiff.imread(image_path)  # Load 3D image stack

        # Transpose the image dimensions to match the expected shape
        image = np.transpose(image, (1, 0, 2, 3))

        # Ensure that there are exactly 2 channels and 15 slices in the stack
        if image.shape != (2, 15, image.shape[2], image.shape[3]):
            print(f"Unsupported image shape: {image.shape}. Skipping {image_file}")
            continue

        # Save each channel as a separate TIFF file using tifffile
        for channel_idx in range(image.shape[0]):
            channel_volume = image[channel_idx, :, :, :]
            channel_name = f"{image_file}_C{channel_idx + 1}"

            # Save the channel as a separate TIFF file using tifffile
            tiff.imsave(os.path.join(output_folder, f"{channel_name}.tif"), channel_volume)


In [ ]:
#this main creates splitted images as output in a new subfolder "splitted images" in the treatment folder
if __name__ == "__main__":
    # Allow the user to choose a folder interactively in Google Colab
    path_input = input_folder_interactive()
    if path_input:
        print(f"Selected folder: {path_input}")

        # Create the output folder path by joining the input folder path with "output"
        path_output = os.path.join(path_input, "output")
        os.makedirs(path_output, exist_ok=True)  # Create the "output" folder if it doesn't exist

        # Create the "splitted images" subfolder within the "output" folder
        path_splitted = os.path.join(path_output, "splitted images")
        os.makedirs(path_splitted, exist_ok=True)  # Create the "splitted images" subfolder if it doesn't exist

        print(f"Output folder created: {path_splitted}")

        # Split channels and save as separate files in the "splitted images" subfolder
        split_and_save_channels(path_input, path_splitted)




